# 0. Setup

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

In [ ]:
%%writefile setup.in
molecule h2o {
  O 
  H 1 0.96
  H 1 0.96 2 104.5
}

set basis cc-pVDZ
energy('scf')

compare_values(-76.0266327341067125, get_variable('SCF TOTAL ENERGY'), 6, 'SCF energy')  #TEST

In [ ]:
!psi4 setup.in

# I. How do we calculate the most accurate energies for a boron atom

lab developed by Tricia D. Shepherd (St. Edwards), Ryan C. Fortanberry (Georgia Southern), Matthew R. Kennedy (Georgia Tech), and C. David Sherrill (Georgia Tech)

### Introduction

The eigenfunction solution to the Schrodinger equation for a multielectron system depends on the coordinates of all electrons. The orbital approximation says that we can represent a many-electron eigenfunction in terms of individual electron orbitals, each of which depends only on the coordinates of a sinlge electron. A _basis set_ in this context is a set of _basis functions_ used to approximate these orbitals. There are two general categories of basis sets: _minimal basis sets_ that describe only occupied orbitals and _extended basis sets_ that describe both occupied and unoccupied orbitals.

### C. How can we improve the accuracy of the HF energy?

To make an even better approximation of our trial function, we may need to take into account polarization effects. (This accounts for the unequal distribution of electrons when two atoms approach each other.) We can include polarization effects by adding STOs of higher orbital angular momentum, _i.e.,_ d-type functions are added to describe valence electrons in 2p orbitals. The presence of poalrization functions is denoted by an asterisk. Two asterisks indicate polarization functions are added to hydrogens. (This would be important if you wanted to account for hydrogen bonding.) An asterisk in parentheses, ``(*)`` indicates that polarization functions are added only to second-row elements.

The presence of diffuse functions is symbolized by the addition of a plus sign to the basis set designator: 6-31+G. Diffuse functions are also useful for systems in an excited stat, for systems with low ionization potential, and systems with some significant negative charge attached. A second +, such as 6-31++G, implies diffuse functions added to hydrogens. The use of double-diffuse basis set is especially useful if you are working with hydrides (H$^-$ ions).

**Q.** Repeat the boron atom energy calculation for each basis set listed below, recording the total HF energy and the number of basis functions used. For each basis, identify the type and number of STO-nG functions used for the core and valence electrons and wheter polarized or diffuse functions are included. Record the total number of orbitals. For the Boron atom, which approximation (choice of basis set) is the most accurate? How does the accuracy relate to the number of basis functions used?

| basis set | HF energy | Number of basis functions |
|---|---|---|
| 6-31G | | |
|6-31+G | | |
| 6-31G* | | |
| 6-31+G* | | |
|6-311G* | | |
|6-311+G** | | |
|cc-pVDZ | | |
|cc-pVTZ | | |

In [ ]:
%%writefile boronC.in
molecule {
    B
}

set reference uhf

for bas in ['6-31g', '6-31+G', '6-31G*', '6-31+G*', '6-311G*', '6-311+G**', 'cc-pVDZ', 'cc-pVTZ']:
    set basis $bas  # special psithon syntax since bas not literal
    
    e, wfn = energy('hf', return_wfn=True)
    print bas, e, wfn.basisset().nbf()
    clean()

In [ ]:
! psi4 boronC.in

### D. How much "correlation energy" can we recover?

Aat the Hartree-Fock level of theory, each electron experiences an average potential field of all the other electrons. In essence, it is a _mean field_ approach that neglects individual electron-electron interactions or _electron correlation_. Thus, we define the difference between the self-consistent field energy and the exact energy as the correlation energy. Two fundametally different approaches to account for electron correlation effects are available by selecting a correlation method: Moller–Plesset (MP) perturbation theory and coupled cluster (CC) theory.

**Q.** By subtracting out the calculated HF energy for the 6-311+G** basis set), determine the value of the correlation energy for boron assuming an "experimental" energy of `-24.608 [Eh]` (Schaefer and Harris, Phys Rev 167, 67 (1968)).

**Q.** Run a MP4 job using a restricted open-shell (ROHF) basis. You'll need to add a keyword/value pair ``qc_module detci`` to access this less common method. From this single job, you can record both MP2 and MP4 energies.

**Q.** Run a CCSD(T) job. From this single submission, you can record both the CCSD and CCSD(T) energies.

| method | total energy | correlation energy | difference from "experiment" | % recovered |
|---|---|---|---|---|
| mp2 | | | | |
| mp4 | | | | |
| ccsd | | | | |
| ccsd(t) | | | | |


**Q.** For each method, determine the percentage of the correlation energy recovered.

In [ ]:
%%writefile boronD.in
molecule {
    B
}

set basis 6-311+G**
set reference rohf
set qc_module detci

energy('mp4')
print_variables()

xptl_tot = -24.608
xptl_corl = xptl_tot - get_variable('hf total energy')
print 'mp2', get_variable('mp2 total energy'), get_variable('mp2 correlation energy'), xptl_tot - get_variable('mp2 total energy'), get_variable('mp2 correlation energy') / xptl_corl
print 'mp4', get_variable('mp4 total energy'), get_variable('mp4 correlation energy'), xptl_tot - get_variable('mp4 total energy'), get_variable('mp4 correlation energy') / xptl_corl 

clean()

set qc_module ccenergy
energy('ccsd(t)')
print_variables()

print 'ccsd', get_variable('ccsd total energy'), get_variable('ccsd correlation energy'), xptl_tot - get_variable('ccsd total energy'), get_variable('ccsd correlation energy') / xptl_corl
print 'ccsd(t)', get_variable('ccsd(t) total energy'), get_variable('ccsd(t) correlation energy'), xptl_tot - get_variable('ccsd(t) total energy'), get_variable('ccsd(t) correlation energy') / xptl_corl 

In [ ]:
! psi4 boronD.in #-o stdout

### E. Can we use DFT (B3LYP) to calculate the electron affinity of boron?

The electron affinity of atom A is the energy released for the process $A + e^- \rightarrow A^-$ or simply the energy difference between the anion and the neutral forms of an atom. Note: electron affinities are reported as positive values.

It was reported (Jursic, Int. J. Quantum Chem., 61, 93 (1997)) that the electron affinity of boron at the B3LYP/6-311+G** level of theory is `0.36 [eV]`. In comparison to the experimental value of `0.28 [eV]`, this led to the assumption that B3LYP does not yield a reasonable electron affinity.

**Q.** Run an unrestricted B3LYP job on neutral boron with the 6-311+G** basis.

**Q.** Set up a job on boron anion by setting charge to ``-1``. Once we do that, the charge and spin multiplicity are no longer compatible. For two electrons in a set of p-orbitals, the multiplicity can only be 3 (triples state, unpaired spins) or 1 (single state, paired spins). Here, by Hund's rules, we expect the spins will remain unpaired, leading to a triplet. Record the energy for boron anion.

**Q.** Calculate the electron affinity. Is this literature result consistent with your calculation? (1 Eh = 27.2116 eV)

**Q.** Repeat the electron affinity calculation of boron, but this time, assume the anion is a singlet state. What is the reason (Bauschlicher, Int. J. Quantum Chem., 66, 285, (1998)) for the reported failure of the B3LYP method?

| boron state | DFT energy | electron affinity |
| --- | --- | --- |
| neutral | | |
| anion (triplet) | | |
| anion (singlet) | | |

In [ ]:
%%writefile boronE.in
molecule {
    0 2
    B
}

set reference uhf
set basis 6-311+G**

e_neu = energy('b3lyp')

molecule batom {
    -1 3
    B
}

e_an_trip = energy('b3lyp')

print e_neu
print e_an_trip, 27.2116 * (e_neu - e_an_trip)

batom.set_molecular_charge(-1)
batom.set_multiplicity(1)

e_an_sing = energy('b3lyp')

print e_an_sing, 27.2116 * (e_neu - e_an_sing)

In [ ]:
! psi4 boronE.in